In [31]:
# Cell 1: Load and prepare data
import pandas as pd
import numpy as np

# Load the cleaned dataset
df = pd.read_csv("era5_cleaned.csv", parse_dates=['valid_time'])

# Ensure chronological order
df = df.sort_values(['longitude', 'latitude', 'valid_time']).reset_index(drop=True)

print(f"Original hourly data shape: {df.shape}")
print(f"Date range: {df['valid_time'].min()} to {df['valid_time'].max()}")
print(f"\nColumns: {df.columns.tolist()}")

Original hourly data shape: (1740960, 12)
Date range: 2025-01-01 00:00:00 to 2025-01-31 23:00:00

Columns: ['valid_time', 'latitude', 'longitude', 't2m', 'd2m', 'msl', 'u10', 'v10', 'tcc', 'skt', 'number', 'expver']


In [32]:
# Cell 2: Create date column
df['date'] = df['valid_time'].dt.date
print(f"Sample of data with date:\n{df[['valid_time', 'date', 'latitude', 'longitude', 't2m']].head()}")

Sample of data with date:
           valid_time        date  latitude  longitude        t2m
0 2025-01-01 00:00:00  2025-01-01      20.0       -9.8  11.005524
1 2025-01-01 01:00:00  2025-01-01      20.0       -9.8  10.411774
2 2025-01-01 02:00:00  2025-01-01      20.0       -9.8   9.693756
3 2025-01-01 03:00:00  2025-01-01      20.0       -9.8   9.174713
4 2025-01-01 04:00:00  2025-01-01      20.0       -9.8   8.739410


In [33]:
# Cell 3: Identify weather parameters to aggregate
# Exclude: coordinate columns, time columns, and metadata columns
exclude_cols = ['valid_time', 'date', 'longitude', 'latitude', 'number', 'expver']
weather_params = [col for col in df.columns if col not in exclude_cols]

print(f"Weather parameters to aggregate ({len(weather_params)}): {weather_params}")
# Should be: t2m, d2m, msl, u10, v10, tcc, skt

Weather parameters to aggregate (7): ['t2m', 'd2m', 'msl', 'u10', 'v10', 'tcc', 'skt']


In [34]:
# Cell 4: Aggregate to daily statistics by location
# Group by date AND spatial coordinates to preserve location information
agg_dict = {}
for param in weather_params:
    agg_dict[param] = ['min', 'max', 'mean']

daily_stats = df.groupby(['date', 'latitude', 'longitude']).agg(agg_dict).reset_index()

# Flatten multi-level columns
daily_stats.columns = ['date', 'latitude', 'longitude'] + \
                       [f'{param}_{stat}' for param in weather_params for stat in ['min', 'max', 'mean']]

print(f"Daily aggregated data shape: {daily_stats.shape}")
print(f"\nColumns created: {daily_stats.columns.tolist()}")
print(f"\nFirst few rows:")
print(daily_stats.head())

Daily aggregated data shape: (72540, 24)

Columns created: ['date', 'latitude', 'longitude', 't2m_min', 't2m_max', 't2m_mean', 'd2m_min', 'd2m_max', 'd2m_mean', 'msl_min', 'msl_max', 'msl_mean', 'u10_min', 'u10_max', 'u10_mean', 'v10_min', 'v10_max', 'v10_mean', 'tcc_min', 'tcc_max', 'tcc_mean', 'skt_min', 'skt_max', 'skt_mean']

First few rows:
         date  latitude  longitude   t2m_min    t2m_max   t2m_mean  d2m_min  \
0  2025-01-01      20.0      -9.80  8.079742  19.427155  13.512838 -2.99888   
1  2025-01-01      20.0      -9.55  7.933746  19.434479  13.462057 -2.94028   
2  2025-01-01      20.0      -9.30  7.940094  19.426666  13.453064 -2.86118   
3  2025-01-01      20.0      -9.05  7.899078  19.409820  13.417399 -2.78550   
4  2025-01-01      20.0      -8.80  7.849762  19.550446  13.468669 -2.77085   

   d2m_max  d2m_mean    msl_min  ...  u10_mean   v10_min   v10_max  v10_mean  \
0 -0.61387 -2.062680  1017.8256  ... -4.516831 -3.625711 -2.103291 -3.016637   
1 -0.58896 -2.064

In [35]:
# Cell 5: Verify aggregation for one location
sample_location = daily_stats.iloc[0]
print(f"Sample location: lat={sample_location['latitude']}, lon={sample_location['longitude']}, date={sample_location['date']}")
print(f"\nExample - t2m aggregation:")
print(f"  t2m_min: {sample_location['t2m_min']:.2f}")
print(f"  t2m_max: {sample_location['t2m_max']:.2f}")
print(f"  t2m_mean: {sample_location['t2m_mean']:.2f}")

Sample location: lat=20.0, lon=-9.8, date=2025-01-01

Example - t2m aggregation:
  t2m_min: 8.08
  t2m_max: 19.43
  t2m_mean: 13.51


In [36]:
# Cell 6: Create next-day target variables (ONLY FOR VARIABLES WE WANT TO PREDICT)
# Sort by location and date to ensure proper shifting
daily_stats = daily_stats.sort_values(['latitude', 'longitude', 'date']).reset_index(drop=True)

# IMPORTANT: Define which variables you want to PREDICT
# Typically, you want to predict temperature (t2m)
# Other variables (d2m, msl, u10, v10, tcc, skt) are just used as FEATURES

variables_to_predict = ['t2m']  # Add more if needed: ['t2m', 'skt', 'tcc']

print(f"Variables to predict: {variables_to_predict}")
print(f"Other variables will be used as features only")

# Create next-day targets ONLY for variables we want to predict
target_cols = []
for var in variables_to_predict:
    for stat in ['min', 'max', 'mean']:
        col_name = f'{var}_{stat}'
        next_col_name = f'{var}_{stat}_next'
        
        # Shift within each location group
        daily_stats[next_col_name] = daily_stats.groupby(['latitude', 'longitude'])[col_name].shift(-1)
        target_cols.append(next_col_name)

print(f"\nCreated {len(target_cols)} next-day target variables:")
print(target_cols)

# Show feature columns vs target columns
feature_cols = [col for col in daily_stats.columns 
                if col not in ['date', 'latitude', 'longitude'] and not col.endswith('_next')]
print(f"\nTotal feature columns: {len(feature_cols)}")
print(f"Target columns: {len(target_cols)}")

Variables to predict: ['t2m']
Other variables will be used as features only

Created 3 next-day target variables:
['t2m_min_next', 't2m_max_next', 't2m_mean_next']

Total feature columns: 21
Target columns: 3


In [37]:
# Cell 7: Remove rows with missing targets and inspect
# Drop rows where any next-day target is NaN (last day for each location)
initial_rows = len(daily_stats)
daily_stats_complete = daily_stats.dropna(subset=target_cols).reset_index(drop=True)
rows_removed = initial_rows - len(daily_stats_complete)

print(f"Rows before: {initial_rows}")
print(f"Rows after removing incomplete: {len(daily_stats_complete)}")
print(f"Rows removed: {rows_removed}")
print(f"\nFirst few rows with features and targets:")
print(daily_stats_complete[['date', 'latitude', 'longitude', 't2m_min', 't2m_max', 't2m_mean', 
                             't2m_min_next', 't2m_max_next', 't2m_mean_next']].head())



###### REMARQUE : DO NOT FORGET TO LINK TO OTHER MONTHS LATER !!!!!!!!!!!!!

Rows before: 72540
Rows after removing incomplete: 70200
Rows removed: 2340

First few rows with features and targets:
         date  latitude  longitude   t2m_min    t2m_max   t2m_mean  \
0  2025-01-01      20.0       -9.8  8.079742  19.427155  13.512838   
1  2025-01-02      20.0       -9.8  8.284332  20.229400  13.733927   
2  2025-01-03      20.0       -9.8  8.742096  20.117584  13.730153   
3  2025-01-04      20.0       -9.8  8.493073  19.262848  13.515086   
4  2025-01-05      20.0       -9.8  9.574616  21.778473  14.880564   

   t2m_min_next  t2m_max_next  t2m_mean_next  
0      8.284332     20.229400      13.733927  
1      8.742096     20.117584      13.730153  
2      8.493073     19.262848      13.515086  
3      9.574616     21.778473      14.880564  
4      8.145905     22.907135      14.977478  


In [38]:
# Cell 8: Verify spatial coverage
unique_locations = daily_stats_complete.groupby(['latitude', 'longitude']).size().reset_index(name='days')
print(f"Number of unique locations: {len(unique_locations)}")
print(f"\nDays per location (should be roughly equal):")
print(unique_locations['days'].describe())
print(f"\nSample locations:")
print(unique_locations.head(10))

Number of unique locations: 2340

Days per location (should be roughly equal):
count    2340.0
mean       30.0
std         0.0
min        30.0
25%        30.0
50%        30.0
75%        30.0
max        30.0
Name: days, dtype: float64

Sample locations:
   latitude  longitude  days
0      20.0      -9.80    30
1      20.0      -9.55    30
2      20.0      -9.30    30
3      20.0      -9.05    30
4      20.0      -8.80    30
5      20.0      -8.55    30
6      20.0      -8.30    30
7      20.0      -8.05    30
8      20.0      -7.80    30
9      20.0      -7.55    30


In [39]:
# Cell 9: Save the engineered features
daily_stats_complete.to_csv("era5_daily_features.csv", index=False)
print("✓ Saved to era5_daily_features.csv")

# Display comprehensive summary
print(f"\n{'='*60}")
print(f"FINAL DATASET SUMMARY")
print(f"{'='*60}")
print(f"Date range: {daily_stats_complete['date'].min()} to {daily_stats_complete['date'].max()}")
print(f"Number of unique locations: {len(unique_locations)}")
print(f"Total daily records: {len(daily_stats_complete)}")
print(f"\nWeather parameters: {weather_params}")
print(f"Features per location per day: {len(feature_cols)} (each parameter has min/max/mean)")
print(f"Target variables: {len(target_cols)}")
print(f"\nColumn breakdown:")
print(f"  - Coordinates: latitude, longitude")
print(f"  - Date: date")
print(f"  - Current day features: {len(feature_cols)}")
print(f"  - Next day targets: {len(target_cols)}")
print(f"  - Total columns: {len(daily_stats_complete.columns)}")

✓ Saved to era5_daily_features.csv

FINAL DATASET SUMMARY
Date range: 2025-01-01 to 2025-01-30
Number of unique locations: 2340
Total daily records: 70200

Weather parameters: ['t2m', 'd2m', 'msl', 'u10', 'v10', 'tcc', 'skt']
Features per location per day: 21 (each parameter has min/max/mean)
Target variables: 3

Column breakdown:
  - Coordinates: latitude, longitude
  - Date: date
  - Current day features: 21
  - Next day targets: 3
  - Total columns: 27


In [40]:
print(daily_stats_complete.columns + "\n")
print(df.columns)

Index(['date\n', 'latitude\n', 'longitude\n', 't2m_min\n', 't2m_max\n',
       't2m_mean\n', 'd2m_min\n', 'd2m_max\n', 'd2m_mean\n', 'msl_min\n',
       'msl_max\n', 'msl_mean\n', 'u10_min\n', 'u10_max\n', 'u10_mean\n',
       'v10_min\n', 'v10_max\n', 'v10_mean\n', 'tcc_min\n', 'tcc_max\n',
       'tcc_mean\n', 'skt_min\n', 'skt_max\n', 'skt_mean\n', 't2m_min_next\n',
       't2m_max_next\n', 't2m_mean_next\n'],
      dtype='object')
Index(['valid_time', 'latitude', 'longitude', 't2m', 'd2m', 'msl', 'u10',
       'v10', 'tcc', 'skt', 'number', 'expver', 'date'],
      dtype='object')
